<a href="https://colab.research.google.com/github/JC1335/projet-ccf/blob/master/Final_Projet_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 **Programmation en RDDs (PySpark) - Python**

In [3]:
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark import SparkConf, SparkContext

# Initialisation de SparkSession et du SparkContext
spark = SparkSession.builder.appName("CCF Correct RDD").getOrCreate()
sc = spark.sparkContext

# Accumulateur pour suivre les nouveaux couples (comme décrit dans le papier)
new_pairs_counter = sc.accumulator(0)

def ccf_correct_implementation(edges_rdd, max_iters=20):
    """
    Implémentation correcte de l'algorithme CCF (Connected Component Finder)
    en utilisant les RDD PySpark, suivant la logique du papier académique.
    """
    global new_pairs_counter

    # Étape 1 : Initialiser chaque nœud avec sa propre étiquette de composante
    nodes = edges_rdd.flatMap(lambda edge: [edge[0], edge[1]]).distinct()
    node_component_rdd = nodes.map(lambda node: (node, node))  # (nœud, id_de_composante)

    # Étape 2 : Créer la liste d’adjacence (bidirectionnelle)
    neighbors_rdd = edges_rdd.flatMap(lambda x: [(x[0], x[1]), (x[1], x[0])])

    iteration = 0
    start_time = time.time()

    # Boucle principale : on propage les composantes jusqu’à convergence ou itérations max
    while iteration < max_iters:
        iteration += 1
        print(f"🔁 Démarrage de l'itération {iteration}...")

        # Réinitialisation de l'accumulateur pour la nouvelle itération
        new_pairs_counter.value = 0

        # 1️ Joindre chaque nœud avec la composante de ses voisins
        joined_rdd = node_component_rdd.join(neighbors_rdd).map(
            lambda x: (x[1][1], x[1][0])  # (voisin, composante du nœud)
        )

        # 2️ Grouper toutes les composantes possibles pour chaque nœud
        input_for_reducer = joined_rdd.union(node_component_rdd).groupByKey()

        # 3️ Phase de réduction (comme décrite dans la Figure 2 du papier)
        def ccf_iterate_reducer(key_values):
            key, values_iter = key_values
            values = list(values_iter)
            min_val = min(values)

            if min_val < key:
                # Ce nœud peut adopter une composante plus petite : on propage
                yield (key, min_val)
                for val in values:
                    if val != min_val:
                        new_pairs_counter.add(1)
                        yield (val, min_val)
            else:
                # Sinon, il garde sa propre étiquette
                yield (key, key)

        # Application du reducer et dédoublonnage
        ccf_iterate_output = input_for_reducer.flatMap(ccf_iterate_reducer)
        dedup_output = ccf_iterate_output.distinct()

        # Mise à jour de la RDD des composantes
        node_component_rdd = dedup_output

        # Action obligatoire pour forcer l’évaluation du compteur (Spark est paresseux)
        node_component_rdd.count()

        # Si aucune propagation n’a eu lieu, on considère qu’on a convergé
        if new_pairs_counter.value == 0:
            print(f"✅ Convergence atteinte en {iteration} itérations.")
            break

    exec_time = time.time() - start_time
    return node_component_rdd, iteration, exec_time

# Chargement des fichiers et exécution du CCF pour chacun ---
schema = StructType([
    StructField("source", IntegerType(), True),
    StructField("target", IntegerType(), True)
])

# Liste des fichiers à analyser
files = [
    ("G1_1k.csv", "G1"),
    ("G2_5k.csv", "G2"),
    ("G3_8k.csv", "G3"),
    ("G4_10k.csv", "G4")
]

# Liste pour stocker les résultats
results = []

for filename, label in files:
    filepath = f"data/{filename}"  # Assure-toi que le fichier est bien dans le dossier "data"
    print(f"📎 Traitement de {label} ({filepath})...")

    try:
        # Chargement du fichier CSV en DataFrame structuré
        df = spark.read.csv(filepath, header=True, schema=schema)

        # Conversion en RDD de paires (source, target)
        edges_rdd = df.rdd.map(lambda row: (row['source'], row['target']))

        # Lancement de l'algorithme CCF
        components, num_iters, exec_time = ccf_correct_implementation(edges_rdd, max_iters=20)

        # 📈 Statistiques du graphe
        nb_nodes = edges_rdd.flatMap(lambda edge: [edge[0], edge[1]]).distinct().count()
        nb_edges = edges_rdd.count()

        print(f"📊 Données du graphe: {nb_nodes} nœuds, {nb_edges} arêtes")
        print(f"🔁 Itérations : {num_iters}")
        print(f"⏱️ Temps : {round(exec_time, 3)} secondes")
        print("-" * 40)

        # Ajout aux résultats
        results.append((label, nb_nodes, nb_edges, num_iters, round(exec_time, 3)))

    except Exception as e:
        print(f"❌ Erreur avec {label} : {e}")
        print("-" * 40)

# Affichage final dans un tableau Pandas
result_rdd_df = pd.DataFrame(
    results,
    columns=["Graphe", "Nœuds", "Arêtes", "Itérations", "Temps (s)"]
)

print("✅ Résumé des performances (RDD) :")
print(result_rdd_df)


📎 Traitement de G1 (data/G1_1k.csv)...
❌ Erreur avec G1 : [PATH_NOT_FOUND] Path does not exist: file:/content/data/G1_1k.csv.
----------------------------------------
📎 Traitement de G2 (data/G2_5k.csv)...
❌ Erreur avec G2 : [PATH_NOT_FOUND] Path does not exist: file:/content/data/G2_5k.csv.
----------------------------------------
📎 Traitement de G3 (data/G3_8k.csv)...
❌ Erreur avec G3 : [PATH_NOT_FOUND] Path does not exist: file:/content/data/G3_8k.csv.
----------------------------------------
📎 Traitement de G4 (data/G4_10k.csv)...
❌ Erreur avec G4 : [PATH_NOT_FOUND] Path does not exist: file:/content/data/G4_10k.csv.
----------------------------------------
✅ Résumé des performances (RDD) :
Empty DataFrame
Columns: [Graphe, Nœuds, Arêtes, Itérations, Temps (s)]
Index: []


# **2 	Implémentation CCF avec DataFrames _ Python**

📎 Traitement de G1 (data/G1_1k.csv)...
❌ Erreur avec G1 : [PATH_NOT_FOUND] Path does not exist: file:/content/data/G1_1k.csv.
----------------------------------------
📎 Traitement de G2 (data/G2_5k.csv)...
❌ Erreur avec G2 : [PATH_NOT_FOUND] Path does not exist: file:/content/data/G2_5k.csv.
----------------------------------------
📎 Traitement de G3 (data/G3_8k.csv)...
❌ Erreur avec G3 : [PATH_NOT_FOUND] Path does not exist: file:/content/data/G3_8k.csv.
----------------------------------------
📎 Traitement de G4 (data/G4_10k.csv)...
❌ Erreur avec G4 : [PATH_NOT_FOUND] Path does not exist: file:/content/data/G4_10k.csv.
----------------------------------------
✅ Résumé des performances (DataFrame) :
Empty DataFrame
Columns: [Graphe, Nœuds, Arêtes, Itérations, Temps (s)]
Index: []
